In [39]:
import cv2
from numpy import *
import numpy as np
import os
import random
import stereo_setting as stset

In [40]:
# Rectifying the images
imgL, imgR = cv2.imread("../test/left/5_L_.png"), cv2.imread("../test/right/5_R_.png")
vert, hori = imgL.shape[:2]
left_stereo_map, right_stereo_map, _ = stset.st_maps("./calibParams/", (hori, vert))
rectL, rectR = stset.st_rectify(imgL, imgR, left_stereo_map, right_stereo_map)

In [41]:
grayR = cv2.cvtColor(rectL,cv2.COLOR_BGR2GRAY)
grayL = cv2.cvtColor(rectR,cv2.COLOR_BGR2GRAY)

# Create StereoSGBM and prepare all parameters
window_size = 1
min_disp = 16*0
num_disp = (16*16)-min_disp
left_matcher = cv2.StereoSGBM_create(minDisparity = min_disp,
    numDisparities = num_disp,
    blockSize = window_size,
    uniquenessRatio = 10,
    speckleWindowSize = 100,
    speckleRange = 32,
    disp12MaxDiff = 5,
    P1 = 8*3*window_size**2,
    P2 = 32*3*window_size**2)

# Used for the filtered image
right_matcher = cv2.ximgproc.createRightMatcher(left_matcher) # Create another stereo for right this time

# WLS FILTER Parameters
lmbda = 80000
sigma = 1.8
visual_multiplier = 1.0

wls_filter = cv2.ximgproc.createDisparityWLSFilter(matcher_left=left_matcher)
wls_filter.setLambda(lmbda)
wls_filter.setSigmaColor(sigma)

dispL = left_matcher.compute(grayL,grayR)
dispR = right_matcher.compute(grayR,grayL)
dispL = np.int16(dispL)
dispR = np.int16(dispR)

filteredImg = wls_filter.filter(dispL, grayL, None, dispR)  # important to put "imgL" here!!!
filteredImg = cv2.normalize(src=filteredImg, dst=filteredImg, beta=0, alpha=255, norm_type=cv2.NORM_MINMAX);
filteredImg = np.uint8(filteredImg)
cv2.imwrite('./filteredImg.png', filteredImg)

fil = cv2.resize(filteredImg, (640, 480))
cv2.imshow('filteredImg',fil)
cv2.waitKey(0)
cv2.destroyAllWindows()